In [1]:
!pip install python-constraint

  Preparing metadata (setup.py) ... done
  Created wheel for python-constraint: filename=python_constraint-1.4.0-py2.py3-none-any.whl size=24059 sha256=3fd487c909469d486a30a7f08460d3df4c7779c5df2c3784e13a26680751aeaa
  Stored in directory: /root/.cache/pip/wheels/2e/f2/2b/cb08b5fe129e4f69b7033061f256e5c551b0aa1160c2872aee
Successfully built python-constraint


In [2]:
from constraint import Problem
import random

In [3]:
# Crear el problema CSP
problem = Problem()

In [4]:
# Definición de aulas, horarios y días
aulas = ["UH45", "UA20", "UC38"]
horarios = ["07:00", "09:00", "11:00", "13:00"]
dias = ["Lunes", "Martes", "Miércoles"]

In [5]:
# Definición de cursos
cursos = ["Introducción a los Algoritmos", "Algoritmos", "Algoritmos y Estructuras de Datos", "Complejidad Algorítmica"]

In [6]:
# Generar capacidad de aulas
capacidad_aulas = {"UH45": 25, "UA20": 30, "UC38": 28}

In [7]:
# Asignar cantidad de estudiantes por curso de manera aleatoria
estudiantes_curso = {curso: random.randint(20, 30) for curso in cursos}
estudiantes_totales = sum(estudiantes_curso.values())
print(f"Estudiantes totales: {estudiantes_totales}")

Estudiantes totales: 95


In [8]:
# Definición del dominio para cada curso (aula, horario, día)
for curso in cursos:
    problem.addVariable(curso, [(aula, horario, dia) for aula in aulas for horario in horarios for dia in dias])

In [9]:
# Restricción: Un curso no puede exceder la capacidad del aula asignada
def verificar_capacidad(asignacion, curso):
    aula, horario, dia = asignacion
    return estudiantes_curso[curso] <= capacidad_aulas[aula]

for curso in cursos:
    problem.addConstraint(lambda asignacion, curso=curso: verificar_capacidad(asignacion, curso), [curso])

In [10]:
# Restricción: Un aula no puede ser asignada a más de un curso en el mismo horario y día
def restriccion_aulas_distintas(asignacion1, asignacion2):
    aula1, horario1, dia1 = asignacion1
    aula2, horario2, dia2 = asignacion2
    return not (aula1 == aula2 and horario1 == horario2 and dia1 == dia2)

for i, curso1 in enumerate(cursos):
    for curso2 in cursos[i + 1:]:
        problem.addConstraint(restriccion_aulas_distintas, (curso1, curso2))

In [11]:
# Buscar todas las soluciones posibles
soluciones = problem.getSolutions()

mejor_solucion = None
max_aulas_diferentes = 0

for solucion in soluciones:
    aulas_utilizadas = {asignacion[0] for asignacion in solucion.values()}
    num_aulas = len(aulas_utilizadas)

    if num_aulas > max_aulas_diferentes:
        max_aulas_diferentes = num_aulas
        mejor_solucion = solucion

In [12]:
if mejor_solucion:
    print(f"Mejor solución encontrada con {max_aulas_diferentes} aulas diferentes:")
    for curso, asignacion in mejor_solucion.items():
        print(f"{curso}: {asignacion}")
else:
    print("No se encontraron soluciones válidas.")

Mejor solución encontrada con 3 aulas diferentes:
Algoritmos y Estructuras de Datos: ('UC38', '13:00', 'Miércoles')
Algoritmos: ('UC38', '13:00', 'Martes')
Complejidad Algorítmica: ('UA20', '13:00', 'Miércoles')
Introducción a los Algoritmos: ('UH45', '13:00', 'Miércoles')
